<a href="https://colab.research.google.com/github/guerrac2001/Colab-Notebooks/blob/main/DataYFinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Permitir a Colab trabajar desde drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Instalar Yahoo Finance para obtener los datos

In [ ]:
pip install yfinance

Instalar Torch y numpy

In [4]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Obtener los precios desde una fecha de inicio hasta el día de hoy

In [5]:
import yfinance as yahooFinance
import datetime
starDate = datetime.datetime (2018, 1, 1)
endDate = datetime.datetime.now()
VALID_START = datetime.datetime (2019,12,31)
dataM = yahooFinance.download("GOOG",start = starDate, end=endDate, interval = "1mo", auto_adjust = True)
valid_size = dataM.loc[VALID_START:endDate].shape[0]

print (dataM)

[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Volume
Date                                                                    
2018-01-01  1048.339966  1186.890015  1045.229980  1169.939941  28738400
2018-02-01  1162.609985  1174.000000   992.559998  1104.729980  42382000
2018-03-01  1107.869995  1177.050049   980.640015  1031.790039  45353300
2018-04-01  1022.820007  1094.165039   990.369995  1017.330017  41715900
2018-05-01  1013.659973  1110.750000  1006.289978  1084.989990  31849400
2018-06-01  1099.349976  1186.286011  1096.010010  1115.650024  32096000
2018-07-01  1099.000000  1273.890015  1093.800049  1217.260010  31940100
2018-08-01  1228.000000  1256.500000  1188.239990  1218.189941  28808400
2018-09-01  1204.270020  1212.989990  1146.910034  1193.469971  28862400
2018-10-01  1199.890015  1209.959961   995.830017  1076.770020  48494700
2018-11-01  1075.800049  1095.569946   996.020020  1094

Obtener los precios de cierre mensuales de la acción 

In [6]:
PreciosCierre = dataM['Close'].values
print(PreciosCierre)

[1169.93994141 1104.72998047 1031.79003906 1017.33001709 1084.98999023
 1115.65002441 1217.26000977 1218.18994141 1193.4699707  1076.77001953
 1094.43005371 1035.60998535 1116.36999512 1119.92004395 1173.31005859
 1188.47998047 1103.63000488 1080.91003418 1216.68005371 1188.09997559
 1219.         1260.10998535 1304.95996094 1337.02001953 1434.22998047
 1339.32995605 1162.81005859 1348.66003418 1428.92004395 1413.60998535
 1482.95996094 1634.18005371 1469.59997559 1621.01000977 1760.73999023
 1751.88000488 1835.73999023 2036.85998535 2068.62988281 2410.12011719
 2411.56005859 2506.32006836 2704.41992188 2909.23999023 2665.31005859
 2965.40991211 2849.04003906 2893.59008789 2713.9699707  2697.82006836
 2792.98999023 2299.33007812 2280.7800293  2187.44995117 2181.62011719]


Permitir que los archivos en drive trabajen dentro de colab

In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


Importar herramientas de Torch

In [8]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import (Dataset, TensorDataset,
 DataLoader, Subset)
import numpy as np
import os
import random

Importar el método para convertir los precios de las acciones en un arreglo usando numpy

In [9]:
import datacnn

Creación de datos en arreglos / convertir en tensores

In [10]:
from datacnn import crear_data_entrada

x, y= crear_data_entrada (PreciosCierre, 1)

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float().unsqueeze(dim=1)

print (x_tensor)
print (x_tensor.shape)
print (y_tensor)
print (y_tensor.shape)

tensor([[1169.9399],
        [1104.7300],
        [1031.7900],
        [1017.3300],
        [1084.9900],
        [1115.6500],
        [1217.2600],
        [1218.1899],
        [1193.4700],
        [1076.7700],
        [1094.4301],
        [1035.6100],
        [1116.3700],
        [1119.9200],
        [1173.3101],
        [1188.4800],
        [1103.6300],
        [1080.9100],
        [1216.6801],
        [1188.1000],
        [1219.0000],
        [1260.1100],
        [1304.9600],
        [1337.0200],
        [1434.2300],
        [1339.3300],
        [1162.8101],
        [1348.6600],
        [1428.9200],
        [1413.6100],
        [1482.9600],
        [1634.1801],
        [1469.6000],
        [1621.0100],
        [1760.7400],
        [1751.8800],
        [1835.7400],
        [2036.8600],
        [2068.6299],
        [2410.1201],
        [2411.5601],
        [2506.3201],
        [2704.4199],
        [2909.2400],
        [2665.3101],
        [2965.4099],
        [2849.0400],
        [2893

Crear dataset para aprendisaje y entrenamiento de red

In [12]:
BATCH_SIZE= 5

dataset = TensorDataset(x_tensor, y_tensor)
valid_ind = len (x) - valid_size

train_dataset = Subset(dataset, list(range(valid_ind)))
valid_dataset = Subset(dataset, list(range(valid_ind, len(x))))

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE)
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE)
